In [1]:
import numpy as np

In [2]:
t = np.load("anis6.npy")
props = np.load("anis6_props.npy")

In [3]:
t.shape

(120, 5000, 3)

In [7]:
props.shape

(1, 5000)

In [8]:
# make the shape of the array possible to save as csv
# t = t.reshape(t.shape[0], -1)

tt = []
for i in range(t.shape[0]):
    # print(t[i].shape, props[i].shape) (5000, 3) (5000,)
    ttt = np.hstack((t[i], props[0][:, None]))
    tt.extend(ttt)
# tt = np.array(tt)

firstline = [5000]*120

#add first line
tt = [firstline] + tt

# save to csv without numpy
with open("drosophila.csv", "w") as f:
    for row in tt:
        f.write(",".join(map(str, row)) + "\n")

In [13]:
def to_decimal_odds(odds_or_probs):
    """If values sum to ~100 treat as percentages and convert to decimal odds (100/p).
       Otherwise assume already decimal odds and return unchanged."""
    s = sum(odds_or_probs)
    if abs(s - 100) < 1e-6:  # detected percentages summing to 100
        return [100.0 / p for p in odds_or_probs]
    return list(odds_or_probs)

def find_arbitrage(book1, book2, total_stake=1.0):
    # convert if needed
    b1 = to_decimal_odds(book1)
    b2 = to_decimal_odds(book2)

    if len(b1) != len(b2):
        raise ValueError("Both bookmakers must offer the same number of outcomes in matching order.")

    # choose best odds per outcome and record which book
    best = []
    for i in range(len(b1)):
        if b1[i] >= b2[i]:
            best.append(("book1", b1[i]))
        else:
            best.append(("book2", b2[i]))

    inv_sum = sum(1.0 / o for _, o in best)

    if inv_sum >= 1.0:
        return {
            "arbitrage": False,
            "inv_sum": inv_sum
        }

    # stakes as fractions of total_stake
    stakes = [((1.0 / o) / inv_sum) * total_stake for _, o in best]
    # guaranteed profit (should be same whichever outcome wins)
    profit = stakes[0] * best[0][1] - total_stake  # absolute profit in same currency as stake
    profit_pct = profit / total_stake * 100.0

    # prepare readable result
    result = {
        "arbitrage": True,
        "inv_sum": inv_sum,
        "bets": [
            {
                "outcome_index": i,
                "place_at": best[i][0],
                "odds": best[i][1],
                "stake": stakes[i],
                "stake_pct_of_total": stakes[i] / total_stake * 100.0
            }
            for i in range(len(best))
        ],
        "guaranteed_profit_absolute": profit,
        "guaranteed_profit_percent": profit_pct
    }
    return result

# ---------- Example ----------
# Book 1 provides decimal odds (3 outcomes)
book1 = [1.66, 4.2, 5.]

# Book 2 we give as percentages that sum to 100 -> auto converted to decimal odds
book2_percent = [60, 24, 19]  # will convert to [100/28, 100/50, 100/22]

# normalize
book2_percent = [x / sum(book2_percent) * 100 for x in book2_percent]

print(book2_percent)
res = find_arbitrage(book1, book2_percent, total_stake=1.0)

if not res["arbitrage"]:
    print("No arbitrage. inv_sum =", res["inv_sum"])
else:
    print(f"Arbitrage! inv_sum = {res['inv_sum']:.6f}")
    for b in res["bets"]:
        print(f"Outcome {b['outcome_index']}: bet {b['stake_pct_of_total']:.2f}% "
              f"on {b['place_at']} at odds {b['odds']}")
    print(f"Guaranteed profit: {res['guaranteed_profit_percent']:.4f}% of total stake "
          f"(absolute profit = {res['guaranteed_profit_absolute']:.6f})")


[58.252427184466015, 23.300970873786408, 18.446601941747574]
No arbitrage. inv_sum = 1.0
